# 🧠 OpenAI Vector Store Setup
Dieses Notebook lädt eine Datei hoch, erstellt einen Vector Store, verbindet ihn mit einem Assistant und ermöglicht die semantische Suche mit GPT-4.

In [ ]:
# 📦 Installiere notwendige Pakete
!pip install openai python-dotenv

In [3]:
# 🔐 API-Key einbinden (über .env Datei empfohlen)
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# 📁 Datei hochladen
from pathlib import Path

file_path = "../data/content.pdf"  # Passe dies an deine Datei an
file = openai.files.create(file=open(file_path, "rb"), purpose="assistants")
print("✅ Datei hochgeladen:", file.id)

✅ Datei hochgeladen: file-LSaayutqGYLqEmUR54M4Ht


In [14]:
# 🧠 Vector Store erstellen
from openai import OpenAI
client = OpenAI()
vector_store = client.vector_stores.create(name="MeinVektorStore")
print("📌 Vector Store ID:", vector_store.id)

📌 Vector Store ID: vs_67f81d72705881918e228b5b84653a26


In [15]:
# ➕ Datei zum Vector Store hinzufügen

vector_store_file = client.vector_stores.files.create(
  vector_store_id="vs_67f81d72705881918e228b5b84653a26",
  file_id="file-LSaayutqGYLqEmUR54M4Ht"
)

#openai.beta.vector_stores.file_batches.upload_and_poll(
#    vector_store_id=vector_store.id,
#    files=[file.id]
#)
print("✅ Datei im Vector Store gespeichert")

✅ Datei im Vector Store gespeichert


In [24]:
# 🤖 Assistant erstellen
assistant = client.beta.assistants.create(
    name="Hunde-KI",
    instructions="Beantworte Fragen basierend auf dem Dokument.",
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {
            "vector_store_ids": ["vs_67f81d72705881918e228b5b84653a26"]
        }
    },
    model="gpt-4o",
)
print("🧠 Assistant-ID:", assistant.id)

🧠 Assistant-ID: asst_gD8e064WP15q6ks3inNaDtW2


In [25]:
# 💬 Thread starten
thread = openai.beta.threads.create()
print("💬 Thread-ID:", thread.id)

💬 Thread-ID: thread_ppB7p30iqrgoouztgaCtRanO


In [26]:
# ❓ Frage stellen und Antwort erhalten
def frage_stellen(frage: str):
    openai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=frage,
    )
    run = openai.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    messages = openai.beta.threads.messages.list(thread_id=thread.id)
    for msg in reversed(messages.data):
        if msg.role == "assistant":
            return msg.content[0].text.value
    return "Keine Antwort gefunden."

# Beispiel-Frage:
frage_stellen("Warum bellt ein Hund?")


'Ein Hund bellt aus verschiedenen Gründen. Ein Hauptgrund ist die Kommunikation mit seinem Umfeld, einschließlich der Kommunikation von Bedürfnissen oder Warnungen. Hunde können auch bellen, um Aufmerksamkeit zu erhalten, aus Unsicherheit oder Angst, als Antwort auf unbekannte Reize oder als Ausdruck von Erregung oder Freude. Bellen kann auch ein Ausdruck territorialen Verhaltens sein, bei dem der Hund auf möglicherweise bedrohliche Bewegungen oder Personen reagiert【4:0†content.pdf】.'

👉 Jetzt kannst du oben deine Datei anpassen und beliebige Fragen stellen.